In [1]:
import pandas as pd
import numpy as np
from dis import dis
import math
import geopandas as gpd
import pandas as pd
import numpy
from shapely import wkt
from shapely import wkb
from shapely.geometry import Point
from geopandas import GeoDataFrame

modified = pd.read_csv('../data/modified_data.csv')
modified_test = pd.read_csv('../data/modified_test_data.csv')

# For train set
geometry_train = [Point(xy) for xy in zip(modified.lat, modified.lon)]
#modified = modified.drop(['lon', 'lat'], axis=1)
gdf_train = GeoDataFrame(modified, crs="EPSG:4326", geometry=geometry_train)

# For test set
geometry_test = [Point(xy) for xy in zip(modified_test.lat, modified_test.lon)]
#modified = modified.drop(['lon', 'lat'], axis=1)
gdf_test = GeoDataFrame(modified_test, crs="EPSG:4326", geometry=geometry_test)

#modified['geometry'] = [Point(xy) for xy in zip(modified.lat, modified.lon)]
#modified[]
#modified['geometry'] = modified['geometry'].replace(' POINT ', 'POINT')


grunnkrets_stripped = pd.read_csv('../data/grunnkrets_norway_stripped.csv')
busstops = pd.read_csv('../data/busstops_norway.csv')
bs = busstops

df_train = gdf_train
df_test = gdf_test


# def dist(lat, lon, df): 
#     lat1 = abs(df['lat'] - lat)
#     lon1 = abs(df['lon'] - lon)
#     eucdist = np.sqrt(lon1**2 + lat1**2)
#     print(type(eucdist))
#     return eucdist


In [2]:
# Calculate distance between points in meters, https://math.stackexchange.com/questions/29157/how-do-i-convert-the-distance-between-two-lat-long-points-into-feet-meters
def dist_to_all_km(lat, lon, df):

    # coordinates in radians
    lat1 = lat*math.pi/180
    lon1 = lon*math.pi/180
    lat2 = df['lat']*math.pi/180 # go through whole lat column
    lon2 = df['lon']*math.pi/180 # go through whole lon column

    # store original coordinates in new dataframe
    distances = pd.DataFrame()
    distances['lat'] = df['lat'].copy()
    distances['lon'] = df['lon'].copy()

    # calculate cartesian coordinates
    R = 6371 # Earth radius in km
    df['x'] = R*np.cos(lat2)*np.cos(lon2)
    df['y'] = R*np.cos(lat2)*np.sin(lon2)
    df['z'] = R*np.sin(lat2)
    x1 = R*np.cos(lat1)*np.cos(lon1)
    y1 = R*np.cos(lat1)*np.sin(lon1)
    z1 = R*np.sin(lat1)

    # calculate distance, store as new column in the distances dataframe
    distances['dist'] = np.sqrt(np.square(df['x']-x1)+np.square(df['y']-y1)+np.square(df['z']-z1))

    return distances['dist'].squeeze()


In [3]:
# import geopandas as gpd
# import shapely.geometry
# from shapely.geometry import Point
# import pandas as pd
# def dist_to_all_meter(lat, lon, data):
#     gdb = gpd.GeoDataFrame(
#         geometry=gpd.GeoSeries.from_wkt(
#             data["geometry"],
#             crs="epsg:4386",
#         )
#     )
#     point = Point(lat, lon)
#     gdb["distance"] = gdb.to_crs(gdb.estimate_utm_crs())["geometry"].distance(
#         gpd.GeoSeries([point for _ in range(len(gdb))], crs=gdb.crs).to_crs(
#             gdb.estimate_utm_crs()
#         )
#     ) 
#     return gdb
# dist_to_all_meter(50, 10, modified)



In [4]:

def store_dist_lvl(distance, df): # enter distance in km
    #distance = 10
    df['num_stores_within_distance_and_same_lvl1'] = 0
    df['num_stores_within_distance_and_same_lvl2'] = 0
    df['num_stores_within_distance'] = 0
    num = 0 

    for index in range(len(df)):
        lat = df._get_value(index,'lat')
        lon = df._get_value(index,'lon')
        lvl1 = df._get_value(index, 'lv1')
        lvl2 = df._get_value(index, 'lv2')
        dist_to_all = dist_to_all_km(lat, lon, df)

        count_distance_and_lvl1 = 0 
        count_distance_and_lvl2 = 0 
        count_distance = 0 #only for distance
        iter = 0
        
        for number in dist_to_all:
            if number < distance:
                count_distance += 1
                if lvl1 == df._get_value(iter, 'lv1'):
                    count_distance_and_lvl1 += 1
                    if lvl2 == df._get_value(iter, 'lv2'):
                        count_distance_and_lvl2 += 1
            iter += 1

        df._set_value(index, 'num_stores_within_distance_and_same_lvl1', count_distance_and_lvl1-1)
        df._set_value(index, 'num_stores_within_distance_and_same_lvl2', count_distance_and_lvl2-1)
        df._set_value(index, 'num_stores_within_distance', count_distance-1)

        num+=1 #for runtime tracking only
        print(str(num) + "/12860")


In [5]:
busstops['geometry'] = gpd.GeoSeries.from_wkt(busstops['geometry'])
busstops['lon'] = busstops.geometry.apply(lambda x: x.x)
busstops['lat'] = busstops.geometry.apply(lambda x: x.y)
def num_busstops_wihin_distance(df): # enter x in km
    df['busstops_within_50m'] = 0
    df['busstops_within_100m'] = 0
    df['busstops_within_400m'] = 0
    df['busstops_within_800m'] = 0
    df['busstops_within_1500m'] = 0
    df['closest_busstop_distance'] = 0

    iter = 0

    for index in range(len(df)):
        one_to_all = dist_to_all_km(df._get_value(index, 'lat'), df._get_value(index, 'lon'), busstops)
        one_to_all = one_to_all.to_frame()
        one_to_all.rename( columns={0 :'a'}, inplace=True)
        count50 = (one_to_all < 0.05).sum()
        count100 = (one_to_all < 0.1).sum()
        count400 = (one_to_all < 0.4).sum()
        count800 = (one_to_all < 0.8).sum()
        count1500 = (one_to_all < 1.5).sum()
        closest = one_to_all.min()
        iter += 1
        print(iter)
        df._set_value(index, 'busstops_within_50m', count50)
        df._set_value(index, 'busstops_within_100m', count100)
        df._set_value(index, 'busstops_within_400m', count400)
        df._set_value(index, 'busstops_within_800m', count800)
        df._set_value(index, 'busstops_within_1500m', count1500)
        df._set_value(index, 'closest_busstop_distance', closest)

num_busstops_wihin_distance(modified)
num_busstops_wihin_distance(modified_test)



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


ValueError: setting an array element with a sequence.

In [ ]:

df.head()

,Unnamed: 0,store_id,grunnkrets_id,lat,lon,mall_dummy,lv1,lv2,total_nbr_people,nbr_people_per_store_in_grunnkrets,...,busstops_within_100m,busstops_within_400m,busstops_within_800m,busstops_within_1500m,num_stores_within_distance_and_same_lvl1,num_stores_within_distance_and_same_lvl2,num_stores_within_distance,x,y,z
0,0,983540538-974187930-44774,6020303.0,59.743104,10.204928,1.0,1,1.1,157.0,3.140000,...,0,0,14,51,46,31,127,3159.422103,568.750298,5503.109673
1,1,987074191-973117734-44755,3010306.0,59.913759,10.734031,0.0,1,1.1,NaN,NaN,...,0,20,40,113,204,157,484,3137.917127,594.846117,5512.646842
2,2,984890265-981157303-64491,6050102.0,60.164751,10.254656,1.0,1,1.1,372.0,26.571429,...,1,9,16,34,14,10,45,3118.990969,564.267188,5526.584739
3,3,914057442-992924179-126912,18040102.0,67.283669,14.379796,1.0,1,1.1,514.0,10.936170,...,2,7,15,46,28,20,89,2383.203886,611.007066,5876.789141
4,4,913018583-913063538-668469,16017414.0,63.358068,10.374832,1.0,1,1.1,NaN,NaN,...,0,14,40,98,6,3,28,2810.134063,514.480268,5694.567376


In [ ]:
store_dist_lvl(1, df_train)

1/12860
2/12860
3/12860
4/12860
5/12860
6/12860
7/12860
8/12860
9/12860
10/12860
11/12860
12/12860
13/12860
14/12860
15/12860
16/12860
17/12860
18/12860
19/12860
20/12860
21/12860
22/12860
23/12860
24/12860
25/12860
26/12860
27/12860
28/12860
29/12860
30/12860
31/12860
32/12860
33/12860
34/12860
35/12860
36/12860
37/12860
38/12860
39/12860
40/12860
41/12860
42/12860
43/12860
44/12860
45/12860
46/12860
47/12860
48/12860
49/12860
50/12860
51/12860
52/12860
53/12860
54/12860
55/12860
56/12860
57/12860
58/12860
59/12860
60/12860
61/12860
62/12860
63/12860
64/12860
65/12860
66/12860
67/12860
68/12860
69/12860
70/12860
71/12860
72/12860
73/12860
74/12860
75/12860
76/12860
77/12860
78/12860
79/12860
80/12860
81/12860
82/12860
83/12860
84/12860
85/12860
86/12860
87/12860
88/12860
89/12860
90/12860
91/12860
92/12860
93/12860
94/12860
95/12860
96/12860
97/12860
98/12860
99/12860
100/12860
101/12860
102/12860
103/12860
104/12860
105/12860
106/12860
107/12860
108/12860
109/12860
110/12860
111/1286

In [ ]:
store_dist_lvl(1, df_test)

1/12860
2/12860
3/12860
4/12860
5/12860
6/12860
7/12860
8/12860
9/12860
10/12860
11/12860
12/12860
13/12860
14/12860
15/12860
16/12860
17/12860
18/12860
19/12860
20/12860
21/12860
22/12860
23/12860
24/12860
25/12860
26/12860
27/12860
28/12860
29/12860
30/12860
31/12860
32/12860
33/12860
34/12860
35/12860
36/12860
37/12860
38/12860
39/12860
40/12860
41/12860
42/12860
43/12860
44/12860
45/12860
46/12860
47/12860
48/12860
49/12860
50/12860
51/12860
52/12860
53/12860
54/12860
55/12860
56/12860
57/12860
58/12860
59/12860
60/12860
61/12860
62/12860
63/12860
64/12860
65/12860
66/12860
67/12860
68/12860
69/12860
70/12860
71/12860
72/12860
73/12860
74/12860
75/12860
76/12860
77/12860
78/12860
79/12860
80/12860
81/12860
82/12860
83/12860
84/12860
85/12860
86/12860
87/12860
88/12860
89/12860
90/12860
91/12860
92/12860
93/12860
94/12860
95/12860
96/12860
97/12860
98/12860
99/12860
100/12860
101/12860
102/12860
103/12860
104/12860
105/12860
106/12860
107/12860
108/12860
109/12860
110/12860
111/1286

In [ ]:
df_train = df_train.drop(columns=df_train.columns[0], axis=1)
df_train = df_train.drop(['grunnkrets_id'], axis=1)
df_train = df_train.drop(['x'], axis=1)
df_train = df_train.drop(['y'], axis=1)
df_train = df_train.drop(['z'], axis=1)
df_train = df_train.drop(['lat'], axis=1)
df_train = df_train.drop(['lon'], axis=1)
df_train = df_train.drop(['geometry'], axis=1)

#meanrev
df_train = df_train.drop(['mean_revenue_for_municipality_and_level1'], axis=1)
df_train = df_train.drop(['mean_revenue_for_municipality_and_level2'], axis=1)
df_train = df_train.drop(['mean_revenue_for_municipality_and_level3'], axis=1)

# For test
df_test = df_test.drop(columns=df_test.columns[0], axis=1)
df_test = df_test.drop(['grunnkrets_id'], axis=1)
df_test = df_test.drop(['x'], axis=1)
df_test = df_test.drop(['y'], axis=1)
df_test = df_test.drop(['z'], axis=1)
df_test = df_test.drop(['lat'], axis=1)
df_test = df_test.drop(['lon'], axis=1)
df_test = df_test.drop(['geometry'], axis=1)

#meanrev
df_test = df_test.drop(['mean_revenue_for_municipality_and_level1'], axis=1)
df_test = df_test.drop(['mean_revenue_for_municipality_and_level2'], axis=1)
df_test = df_test.drop(['mean_revenue_for_municipality_and_level3'], axis=1)

col_at_end = ['revenue']
df_train = df_train[[c for c in df_train if c not in col_at_end] + [c for c in col_at_end if c in df_train]]

df_train.to_csv('train_set.csv', index=False)
df_test.to_csv('test_set.csv', index=False)
df_test.head()

KeyError: "['grunnkrets_id'] not found in axis"

In [ ]:
#df_test = df_test.drop(['mean_revenue_for_municipality_and_level1'], axis=1)
#df_test = df_test.drop(['mean_revenue_for_municipality_and_level2'], axis=1)
#df_test = df_test.drop(['mean_revenue_for_municipality_and_level3'], axis=1)
#df_train = df_train.drop(['mean_revenue_for_municipality_and_level1'], axis=1)
#df_train = df_train.drop(['mean_revenue_for_municipality_and_level2'], axis=1)
#df_train = df_train.drop(['mean_revenue_for_municipality_and_level3'], axis=1)

col_at_end = ['revenue']
df_train = df_train[[c for c in df_train if c not in col_at_end] + [c for c in col_at_end if c in df_train]]

df_train.to_csv('train_set.csv', index=False)
df_test.to_csv('test_set.csv', index=False)
df_test.head()


,store_id,mall_dummy,lv1,lv2,total_nbr_people,nbr_people_per_store_in_grunnkrets,counts_gr_lv2,counts_municipality_lv2,busstops_within_50m,busstops_within_100m,busstops_within_400m,busstops_within_800m,busstops_within_1500m,num_stores_within_distance_and_same_lvl1,num_stores_within_distance_and_same_lvl2,num_stores_within_distance
0,914206820-914239427-717245,0.0,1,1.1,960.0,960.000000,2.0,924.0,2,2,7,31,110,18,14,36
1,916789157-916823770-824309,0.0,1,1.1,NaN,NaN,0.0,36.0,0,1,4,12,52,0,0,5
2,913341082-977479363-2948,1.0,1,1.1,2421.0,127.421053,8.0,924.0,0,0,2,12,44,1,1,7
3,889682582-889697172-28720,0.0,1,1.1,149.0,74.500000,0.0,196.0,0,0,4,26,82,0,0,3
4,997991699-998006945-417222,0.0,1,1.1,4921.0,492.100000,4.0,924.0,0,1,6,35,117,14,10,34
